<a href="https://colab.research.google.com/github/Neethu0207/Recommend-System/blob/main/SkinRecum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing libraries**

In [ ]:

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio
import seaborn as sns
import missingno as mno
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

from bokeh.io import curdoc, push_notebook, output_notebook
from bokeh.layouts import column, layout
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput, HoverTool
from bokeh.plotting import figure, show


**Loading the dataset**

In [ ]:
df=pd.read_csv('/content/MP-Skin Care Product Recommendation System3.csv')

**Data Exploration**

In [ ]:
df.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=..."
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f..."


In [ ]:
df.shape

(1224, 9)

In [ ]:
df.size

11016

In [ ]:
df.dtypes

,0
product_href,object
product_name,object
product_type,object
brand,object
notable_effects,object
skintype,object
price,object
description,object
picture_src,object


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   product_href     1224 non-null   object
 1   product_name     1224 non-null   object
 2   product_type     1224 non-null   object
 3   brand            1224 non-null   object
 4   notable_effects  1224 non-null   object
 5   skintype         1224 non-null   object
 6   price            1224 non-null   object
 7   description      1224 non-null   object
 8   picture_src      1224 non-null   object
dtypes: object(9)
memory usage: 86.2+ KB


In [ ]:
df.isnull().sum()

,0
product_href,0
product_name,0
product_type,0
brand,0
notable_effects,0
skintype,0
price,0
description,0
picture_src,0


In [ ]:

df.duplicated().sum()

np.int64(14)

In [ ]:
pro = df[['brand','product_name','product_type','notable_effects','price']]
pro.describe()

,brand,product_name,product_type,notable_effects,price
count,1224,1224,1224,1224,1224
unique,211,1192,5,151,512
top,SOMETHINC,Somethinc RESURRECT Multibiome Serum,Serum,"Pore-Care, Brightening, Anti-Aging",Rp 89.000
freq,80,4,307,150,31


**From the above info**
1. In total, there are 1224 products scrapped from websites.
2. Unfortunately, there are 14 duplicate rows. Need to be removed.
Of 5 types of products, serum is more hype than others.
3. SOMETHINC is the top brand, which means they have many kinds of products.
4. From many pairs of notable effects, 150 products are good for pore care, brightening, and anti-aging all in one product.
5. Looks like many skin care products suitable for oily skin.

In [ ]:
count_brand=df['brand'].value_counts()
count_percentage=df['brand'].value_counts(1)*100

count_dfbrand_=pd.DataFrame({'Brand':count_brand.index,'Counts':count_brand.values,'Percent%':np.round(count_percentage.values,2)})
top_10_brands = count_dfbrand_.head(10)
top_10_brands


,Brand,Counts,Percent%
0,SOMETHINC,80,6.54
1,WARDAH,69,5.64
2,INNISFREE,49,4.00
3,AVOSKIN,39,3.19
4,ERHA,33,2.70
5,ELSHE SKIN,24,1.96
6,THE BODY SHOP,23,1.88
7,BANANA BOAT,22,1.80
8,AZARINE,20,1.63
9,NATURE REPUBLIC,18,1.47


In [ ]:
count_brand=df['brand'].value_counts()
count_percentage=df['brand'].value_counts()*100

count_dfbrand_=pd.DataFrame({'Brand':count_brand.index,'Counts':count_brand.values,'Percent%':np.round(count_percentage.values,2)})
top_10_brands = count_dfbrand_.head(10)
top_10_brands


,Brand,Counts,Percent%
0,SOMETHINC,80,8000
1,WARDAH,69,6900
2,INNISFREE,49,4900
3,AVOSKIN,39,3900
4,ERHA,33,3300
5,ELSHE SKIN,24,2400
6,THE BODY SHOP,23,2300
7,BANANA BOAT,22,2200
8,AZARINE,20,2000
9,NATURE REPUBLIC,18,1800


In [ ]:
#visualising too 10 brands
fig = px.bar(top_10_brands, x='Brand', y='Counts', color='Brand',
             color_discrete_sequence=px.colors.qualitative.Plotly,
             title='Top 10 Brands')
fig.show()

In [ ]:
#checking product type
count_product=df['product_type'].value_counts()
print(count_product)

product_type
Serum          307
Toner          255
Moisturizer    248
Sunscreen      212
Face Wash      202
Name: count, dtype: int64


In [ ]:
#calculating product type percentage
count_percentage=df['product_type'].value_counts(1)*100
print(count_percentage)

product_type
Serum          25.081699
Toner          20.833333
Moisturizer    20.261438
Sunscreen      17.320261
Face Wash      16.503268
Name: proportion, dtype: float64


In [ ]:
#visualising product type using piechart
fig = px.pie(df, names='product_type', title='Product Type Distribution')
fig.show()

In [ ]:
#visualsing procuct type using histogram
fig = px.histogram(df, x='product_type', title='Product Type Distribution')
fig.show()

In [ ]:
#calculating product name
count_productnam=df['product_name'].value_counts()
print(count_productnam)

product_name
Somethinc RESURRECT Multibiome Serum                   4
Somethinc Skin Defender Bakuchiol + COQ10 Serum        4
Somethinc 5% Niacinamide Barrier Serum                 3
Dear Me Beauty Watermelon Multipurpose Gel             3
Innisfree Retinol Cica Moisture Recovery Serum 30ml    3
                                                      ..
GENTLE HOUR Dew It Again Hydrating Toner               1
GENTLE HOUR Brighter Days Exfoliating Toner            1
Gentle Hour Clean Start Hydrating Jelly Cleanser       1
Garnier Sakura Night Cream                             1
GLAMGLOW YOUTHPOTION Rejuvinating Peptida Serum        1
Name: count, Length: 1192, dtype: int64


In [ ]:
#calculating no of counts of each product name
count_productnam=df['product_name'].value_counts(1)*100
print(count_productnam)

product_name
Somethinc RESURRECT Multibiome Serum                   0.326797
Somethinc Skin Defender Bakuchiol + COQ10 Serum        0.326797
Somethinc 5% Niacinamide Barrier Serum                 0.245098
Dear Me Beauty Watermelon Multipurpose Gel             0.245098
Innisfree Retinol Cica Moisture Recovery Serum 30ml    0.245098
                                                         ...   
GENTLE HOUR Dew It Again Hydrating Toner               0.081699
GENTLE HOUR Brighter Days Exfoliating Toner            0.081699
Gentle Hour Clean Start Hydrating Jelly Cleanser       0.081699
Garnier Sakura Night Cream                             0.081699
GLAMGLOW YOUTHPOTION Rejuvinating Peptida Serum        0.081699
Name: proportion, Length: 1192, dtype: float64


In [ ]:
#selecting top 5 product names used
top_5_product_names = count_productnam.head(5)
print(top_5_product_names)

product_name
Somethinc RESURRECT Multibiome Serum                   0.326797
Somethinc Skin Defender Bakuchiol + COQ10 Serum        0.326797
Somethinc 5% Niacinamide Barrier Serum                 0.245098
Dear Me Beauty Watermelon Multipurpose Gel             0.245098
Innisfree Retinol Cica Moisture Recovery Serum 30ml    0.245098
Name: proportion, dtype: float64


In [ ]:
#skin type that suitable for the product
count=df['skintype'].value_counts()
count_percentage=df['skintype'].value_counts(1)*100
count_df=pd.DataFrame({'Skin Type':count.index,'Counts':count.values,'Percent%':np.round(count_percentage.values,2)})
count_df

,Skin Type,Counts,Percent%
0,Oily,310,25.33
1,"Normal, Dry, Oily, Combination, Sensitive",263,21.49
2,Dry,167,13.64
3,"Normal, Dry",93,7.60
4,"Normal, Dry, Oily, Combination",68,5.56
5,Sensitive,55,4.49
6,"Normal, Dry, Oily, Sensitive",52,4.25
7,"Oily, Combination, Sensitive",43,3.51
8,"Dry, Oily, Sensitive",36,2.94
9,"Dry, Sensitive",27,2.21


In [ ]:
#visualising using piechart
fig = px.pie(df, names='skintype', title='Skin Type Distribution')
fig.show()

In [ ]:
# Creating numerical labels of notable_effects due to many effects shown by the diagram above
df['notable_effects']=df['notable_effects'].astype('category')
df['notable_effects_label']=df['notable_effects'].cat.codes
df.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,notable_effects_label
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,18
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,127
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",127
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,18
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",40


In [ ]:
#skin that siutable for
count=df['notable_effects'].value_counts()
count_percentage=df['notable_effects'].value_counts(1)*100
count_df=pd.DataFrame({'notable_effects':count.index,'Counts':count.values,'Percent%':np.round(count_percentage.values,2)})
count_df

,notable_effects,Counts,Percent%
0,"Pore-Care, Brightening, Anti-Aging",150,12.25
1,Anti-Aging,105,8.58
2,"Acne-Free, Oil-Control, Pore-Care",86,7.03
3,"Moisturizing, Brightening, Black-Spot",71,5.80
4,"Hydrating, Moisturizing",70,5.72
...,...,...,...
146,"Soothing, Moisturizing, Anti-Aging",1,0.08
147,"Soothing, Refreshing, Oil-Control",1,0.08
148,"Soothing, Refreshing",1,0.08
149,"Soothing, Oil-Control, Pore-Care",1,0.08


In [ ]:
#visualising top  notable_effects using histogram
fig = px.histogram(df, x='notable_effects', title='Notable Effects Distribution')
fig.show()

In [ ]:
#visualising only top 5 notable effects using barplot
top_5_effects = count_df.head(5)
fig = px.bar(top_5_effects, x='notable_effects', y='Counts', color='notable_effects',
             color_discrete_sequence=px.colors.qualitative.Plotly,
             title='Top 5 Notable Effects')
fig

Data Preparation

In [ ]:
# Data Preparation
df = pd.read_csv('/content/MP-Skin Care Product Recommendation System3.csv')

from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

if 'skintype' in df.columns:
    df['skintype'] = df['skintype'].astype(str).fillna('').str.split(',')
    df['skintype'] = df['skintype'].apply(lambda x: [item.strip() for item in x if item.strip()])

    mlb = MultiLabelBinarizer()
    skintype_encoded = pd.DataFrame(mlb.fit_transform(df['skintype']),
                                     columns=mlb.classes_,
                                     index=df.index)
    df = df.drop('skintype', axis=1).join(skintype_encoded)


    print("DataFrame head after skintype encoding:")
    display(df.head())

else:
    print("Skipping skintype encoding: 'skintype' column not found.")
    print("Current DataFrame head:")
    display(df.head())

DataFrame head after skintype encoding:


,product_href,product_name,product_type,brand,notable_effects,price,description,picture_src,Combination,Dry,Normal,Oily,Sensitive
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,0,0,0,1,0
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,1,1,1,0,0
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",1,1,1,1,1
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,0,0,0,1,0
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",0,1,1,0,0


In [ ]:
#saving into csv file
df.to_csv('MP-Skin Care Product Recommendation System3.csv', index=False)

# TF-IDF

The TF-IDF Vectorizer will be used in the recommendation system to find a representation of the important features of each notable_effects category. We will use the tfidfvectorizer() function from the sklearn library.

In [ ]:
#initialzing tf-idf
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
#perform idf on notable_effects
tfidf_matrix = tfidf.fit_transform(df['notable_effects'])

In [ ]:
#getting features name out
tfidf.get_feature_names_out()

array(['acne', 'aging', 'anti', 'balancing', 'barrier', 'black',
       'brightening', 'care', 'control', 'free', 'hydrating',
       'moisturizing', 'oil', 'pore', 'protection', 'refreshing', 'skin',
       'soothing', 'spot', 'uv', 'whitecast'], dtype=object)

In [ ]:
#initialzing tf-idf
tfidf = TfidfVectorizer(stop_words='english')

#perform idf on notable_effects
tfidf_matrix = tfidf.fit_transform(df['notable_effects'])

#getting features name out
tfidf.get_feature_names_out()

#converting into matrix format
# The user mistakenly used the alias 'tf' (tensorflow) instead of the 'tfidf' variable.
# Correcting this to use the initialized TfidfVectorizer.
tfidf_matrix = tfidf.fit_transform(df['notable_effects'])

In [ ]:
#viewing matrix size
shape=tfidf_matrix.shape
print(shape)

(1224, 21)


In [ ]:
#convert tfidf into matrix
tfidf_matrix.toarray()

array([[0.44842197, 0.32844212, 0.32844212, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.41403597, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.42500873, 0.42500873, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:

pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf.get_feature_names_out(),
    index=df['product_name']
).sample(shape[1],axis=1).sample(10,axis=0)

,pore,soothing,free,care,skin,refreshing,whitecast,black,hydrating,spot,...,control,oil,aging,protection,uv,brightening,barrier,acne,anti,moisturizing
product_name,,,,,,,,,,,,,,,,,,,,,
COSRX Hydrium Watery Toner,0.000000,0.000000,0.528341,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.386371,0.000000,0.000000,0.379501,0.000000,0.527512,0.386371,0.0
HADA LABO Tamagohada Mild Peeling Face Wash 50gr,0.325576,0.000000,0.392642,0.325576,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.489973,0.489973,0.000000,0.000000,0.000000,0.000000,0.000000,0.392026,0.000000,0.0
Dear Klairs Freshly Juiced Vitamin E Mask,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.707107,0.707107,0.000000,0.000000,0.000000,0.000000,0.0
HAPLE Moonlight Glow (toner AHA),0.481907,0.000000,0.000000,0.481907,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.425009,0.000000,0.000000,0.417452,0.000000,0.000000,0.425009,0.0
Somethinc SOS Bakuchiol Electrolyte Rich Moisturizer Silk Creme,0.530327,0.000000,0.000000,0.530327,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.467711,0.000000,0.000000,0.000000,0.000000,0.000000,0.467711,0.0
Mediheal Tea Tree Biome Blemish Cica Cream,0.372413,0.000000,0.449127,0.372413,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.328442,0.000000,0.000000,0.322602,0.000000,0.448422,0.328442,0.0
Wardah Acnederm Pure Foaming Cleanser,0.451557,0.000000,0.544574,0.451557,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.543720,0.000000,0.0
SCARLETT WHITENING Brightly Ever After Serum,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.580772,0.000000,0.000000,0.570445,0.000000,0.000000,0.580772,0.0
L'ESSENTIAL Triple Cream,0.481907,0.000000,0.000000,0.481907,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.425009,0.000000,0.000000,0.417452,0.000000,0.000000,0.425009,0.0


In [ ]:
#making dataframe to see tfidf matrix
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
df_tfidf

,acne,aging,anti,balancing,barrier,black,brightening,care,control,free,...,moisturizing,oil,pore,protection,refreshing,skin,soothing,spot,uv,whitecast
0,0.448422,0.328442,0.328442,0.000000,0.0,0.0,0.322602,0.372413,0.0,0.449127,...,0.000000,0.0,0.372413,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,0.000000,0.000000,0.000000,0.820405,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.571783,0.0,0.000000,0.0
2,0.000000,0.000000,0.000000,0.820405,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.571783,0.0,0.000000,0.0
3,0.448422,0.328442,0.328442,0.000000,0.0,0.0,0.322602,0.372413,0.0,0.449127,...,0.000000,0.0,0.372413,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.559172,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.829052,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.426141,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.639689,0.000000,0.0,0.000000,0.0,0.639689,0.0
1220,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.421956,0.0,0.000000,0.000000,0.906616,0.0,0.000000,0.0,0.000000,0.0
1221,0.414036,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.414687,...,0.000000,0.0,0.000000,0.000000,0.810314,0.0,0.000000,0.0,0.000000,0.0
1222,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.630729,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [ ]:
#calculating coisine similarity
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.37190959, 0.        ,
        0.77278913],
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.37190959, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.77278913, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:

# Creating a dataframe from the cosine_similarity variable with rows and columns in the form of product names
cosine_sim_df = pd.DataFrame(cosine_sim, index=df['product_name'], columns=df['product_name'])
cosine_sim_df

product_name,ACWELL Bubble Free PH Balancing Cleanser,ACWELL pH Balancing Soothing Cleansing Foam,Acwell Licorice pH Balancing Cleansing Toner,ACWELL Aquaseal Soothing Tonic,Licorice pH Balancing Essence Mist,Licorice pH Balancing Advance Serum,Licorice pH Balancing Essential Cream,Real Aqua Balancing Cream,AHC Peony Bright Clearing Toner,AHC Hyaluronic Toner,...,Whitelab Brightening Night Cream,Whitelab Brightening Facial Wash,Whitelab Brightening Face Serum - Niacinamide 10%,Whitelab Brightening Day Cream,Whitelab Brightening Night Cream,Whitelab Brightening Day Cream,YOU The Radiance White Purifying Facial Foam,YOU Basic Skin Care Simply Fresh and Bright Facial Wash,YOU Dazzling Glow Up Facial Foam,YOU Golden Age Deep Cleansing Facial Wash
product_name,,,,,,,,,,,,,,,,,,,,,
ACWELL Bubble Free PH Balancing Cleanser,1.000000,0.000000,0.000000,1.000000,0.180390,0.458453,0.199951,0.000000,0.772789,0.464487,...,0.102696,1.000000,0.565527,0.102696,0.102696,0.137474,0.000000,0.371910,0.00000,0.772789
ACWELL pH Balancing Soothing Cleansing Foam,0.000000,1.000000,1.000000,0.000000,0.474038,0.000000,0.000000,0.596687,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Acwell Licorice pH Balancing Cleansing Toner,0.000000,1.000000,1.000000,0.000000,0.474038,0.000000,0.000000,0.596687,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
ACWELL Aquaseal Soothing Tonic,1.000000,0.000000,0.000000,1.000000,0.180390,0.458453,0.199951,0.000000,0.772789,0.464487,...,0.102696,1.000000,0.565527,0.102696,0.102696,0.137474,0.000000,0.371910,0.00000,0.772789
Licorice pH Balancing Essence Mist,0.180390,0.474038,0.474038,0.180390,1.000000,0.258584,0.346578,0.000000,0.233427,0.000000,...,0.178004,0.180390,0.318977,0.178004,0.178004,0.238286,0.000000,0.000000,0.00000,0.233427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Whitelab Brightening Day Cream,0.137474,0.000000,0.000000,0.137474,0.238286,0.197065,0.264125,0.000000,0.177893,0.000000,...,0.135656,0.137474,0.243090,0.135656,0.135656,1.000000,0.000000,0.000000,0.00000,0.177893
YOU The Radiance White Purifying Facial Foam,0.000000,0.000000,0.000000,0.000000,0.000000,0.247059,0.331132,0.182655,0.000000,0.000000,...,0.170071,0.000000,0.000000,0.170071,0.170071,0.000000,1.000000,0.734644,0.26614,0.000000
YOU Basic Skin Care Simply Fresh and Bright Facial Wash,0.371910,0.000000,0.000000,0.371910,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.371910,0.000000,0.000000,0.000000,0.000000,0.734644,1.000000,0.00000,0.000000


In [ ]:
#checking similarity with each product
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)


product_name,Everwhite Acnepair Toner,BIO BEAUTY LAB Acne Treatment Face Oil,Mamonde Everyday Sun Stick 50+++++,Lagom Cellus Sun Gel,AZARINE Easy White Nutrifull Night Cream
product_name,,,,,
BIODERMA Photoderm Max fluide SPF100 40ml,0.000000,0.000000,0.827608,0.530562,0.337375
Mediheal Tea Tree Biome Blemish Cica Cream,0.594637,0.301309,0.000000,0.000000,0.772789
Innisfree Soybean Energy Skin EX 200ml,0.000000,0.000000,0.000000,0.000000,0.000000
Whitelab Brightening Facial Wash,0.594637,0.301309,0.000000,0.000000,0.772789
PURIVERA BOTANICALS Sugar Willow Serum - Fruit AHA 10% - Willow BHA 2% - Natural Exfoliate - Peeling - Exfoliating,0.496176,0.723345,0.000000,0.000000,0.299504
SIMPLISTIC Cleanser begin,0.333120,0.000000,0.000000,0.000000,0.493073
SUKIN Signature Original Hydrating Mist Toner,0.000000,0.000000,0.000000,0.000000,0.601053
Innisfree Jeju Cherry Blossom Mist 120 mL,0.000000,0.000000,0.000000,0.000000,0.233427
AVOSKIN YOUR SKIN BAE SERIES Panthenol 5% + Mugwort + Cica Serum,0.000000,0.000000,0.000000,0.000000,0.231080


# Getting Recommmendation


In [ ]:
def skincare_recommendation(product_name, similarity_data=cosine_sim_df, items=df[['product_name', 'brand', 'product_type', 'notable_effects', 'price']]):
    index = similarity_data.loc[:,product_name].to_numpy().argpartition(range(-1,-k,-1))
    #retriving data with greatest similarity
    closest=similarity_data.columns[index[-1:-(k+2):-1]]
    closest=closest.drop(product_name,errors='ignore')
    # Drop a name so that the name of the product we are looking for doesnt' appear in the list of recommendations
    closest=closest.drop(product_name,errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
df[df.product_name.eq('ELSHE SKIN Radiant Supple Serum')].head()


,product_href,product_name,product_type,brand,notable_effects,price,description,picture_src,Combination,Dry,Normal,Oily,Sensitive
292,https://www.beautyhaul.com/product/detail/radi...,ELSHE SKIN Radiant Supple Serum,Serum,ELSHE SKIN,"Acne-Free, Brightening, Black-Spot",Rp 182.750,ElsheSkin Radiant Supple Serum atau Radiant Sk...,https://www.beautyhaul.com/assets/uploads/prod...,1,1,1,1,1


In [ ]:
def skincare_recommendation(product_name, k, similarity_data=cosine_sim_df, items=df[['product_name', 'brand', 'product_type', 'notable_effects', 'price']]):
    index = similarity_data.loc[:,product_name].to_numpy().argpartition(range(-1,-k,-1))
    #retriving data with greatest similarity
    closest=similarity_data.columns[index[-1:-(k+2):-1]]
    closest=closest.drop(product_name,errors='ignore')
    # Drop a name so that the name of the product we are looking for doesnt' appear in the list of recommendations
    closest=closest.drop(product_name,errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)
skincare_recommendation('ELSHE SKIN Radiant Supple Serum', k=5)

,product_name,brand,product_type,notable_effects,price
0,SOMETHINC Dark Spot Reducer Ampoule,SOMETHINC,Serum,"Acne-Free, Brightening, Black-Spot",Rp 129.000
1,AVOSKIN YOUR SKIN BAE SERIES Niacinamide 12% +...,AVOSKIN,Serum,"Hydrating, Acne-Free, Brightening, Black-Spot",Rp 125.100
2,Mineral Botanica Perfect Purifying Acne Night ...,MINERAL BOTANICA,Moisturizer,"Moisturizing, Acne-Free, Black-Spot",Rp 52.000
3,Wardah Crystal Secret Dark Spot & Brightening ...,WARDAH,Serum,"Moisturizing, Brightening, Black-Spot",Rp 60.100
4,Laneige LNG White Dew Emulsion 100ml (OL21),LANEIGE,Moisturizer,"Moisturizing, Brightening, Black-Spot",Rp 566.714


In [ ]:
df[df.product_name.eq('SOMETHINC Dark Spot Reducer Ampoule')].head()

,product_href,product_name,product_type,brand,notable_effects,price,description,picture_src,Combination,Dry,Normal,Oily,Sensitive
1029,https://www.beautyhaul.com/product/detail/dark...,SOMETHINC Dark Spot Reducer Ampoule,Serum,SOMETHINC,"Acne-Free, Brightening, Black-Spot",Rp 129.000,"Solusi bekas jerawat menghitam, flek, dan hipe...",https://www.beautyhaul.com/assets/uploads/prod...,0,1,0,0,1


In [ ]:
skincare_recommendation('SOMETHINC Dark Spot Reducer Ampoule', k=5)

,product_name,brand,product_type,notable_effects,price
0,ELSHE SKIN Radiant Supple Serum,ELSHE SKIN,Serum,"Acne-Free, Brightening, Black-Spot",Rp 182.750
1,AVOSKIN YOUR SKIN BAE SERIES Niacinamide 12% +...,AVOSKIN,Serum,"Hydrating, Acne-Free, Brightening, Black-Spot",Rp 125.100
2,Mineral Botanica Perfect Purifying Acne Night ...,MINERAL BOTANICA,Moisturizer,"Moisturizing, Acne-Free, Black-Spot",Rp 52.000
3,Wardah Crystal Secret Dark Spot & Brightening ...,WARDAH,Serum,"Moisturizing, Brightening, Black-Spot",Rp 60.100
4,Laneige LNG White Dew Emulsion 100ml (OL21),LANEIGE,Moisturizer,"Moisturizing, Brightening, Black-Spot",Rp 566.714
